In [1]:
# requires installing shapely==1.7.1, turfpy, geopandas, pygeos

In [30]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

In [3]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [4]:
def point_to_km(lng, lat, company):
    point = Point(lng, lat, srid=4326)
    roads = Road.objects.filter(company=company).annotate(distance=Distance(point, 'path')).order_by('distance')
    road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [1]:
company=Company.objects.get(name='Caminhos da Serra Gaúcha')
company

<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>

In [2]:
user=User.objects.get(username="rlcs")

In [28]:
reps = Reporting.objects.filter(company=company,end_km__isnull=False)
reps.count()

1259

In [23]:
point_to_km(reps[0].point.x, reps[0].point.y,company)[3]

615.292

In [24]:
reps[0].end_km - reps[0].km + point_to_km(reps[0].point.x, reps[0].point.y,company)[3]

615.332

In [32]:
rep_edit=[]
for rep in tqdm(reps):
    try:
        start = point_to_km(rep.point.x,rep.point.y,company)[3]
        end = rep.end_km - rep.km + start
        rep.km = start
        rep.end_km = end
        rep_edit.append(rep)
    except Exception as e:
        print(e)

  0%|          | 0/1259 [00:00<?, ?it/s]

In [34]:
apont = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=200,user=user)